# Send emails with flask-mail
### Before anything: install flask-mail

### First
You need to set a bunch of config in your app  


In [ ]:
app.config['MAIL_SERVER']     = 'smtp.gmail.com'
app.config['MAIL_PORT']       = 587
app.config['MAIL_USE_TLS']    = True
app.config['MAIL_USE_SSL']    = False
app.config['MAIL_USERNAME']   = 'awetandtesfit@gmail.com'
app.config['MAIL_PASSWORD']   = 'Micheal79'

> You need to allow less secure apps on your google account for this to work, to do so, follow this link:
> https://myaccount.google.com/lesssecureapps


# Create an email manager with flask_mail

flask_mail provide a `Mail` class that allow you to send mails.<br>

<small>In the __init__.py</small>

In [ ]:
# ...
import flask_mail

# ...

app = flask.Flask(__name__)
# define the mail object
mail = flask_mail.Mail(app)

# ...

# Send an email

To send an email, you first need to create the message object

In [ ]:
msg = flask_mail.Message('subject', 
                         sender='mymail@gmail.com', 
                         recipients=['receiver1@example.com', 
                                     'receiver2@example.com'])

And then add some content to the mail, you can add text content or HTML content

In [ ]:
msg.body = 'text body'
msg.html = '<h1>HTML body</h1>'

To send the message, you need to use the `send` function of the mail manager

In [ ]:
mail.send(msg)

We can add every mailing functions in an `mailing.py` file.

In [ ]:
def send_email(subject, sender, recipients, text_body, html_body):
    msg = Message(subject, sender=sender, recipients=recipients)
    msg.body = text_body
    msg.html = html_body
    mail.send(msg)
    

> You can render a template and send it by email

# Add a "lost password" function

### First: let's create a function to send the email
We could create a function like this:

In [ ]:
def send_password_reset_email(user):
    send_email('Password reset',
               sender="mymail@gmail.com",
               recipients=[user.email],
               text_body=flask.render_template('email/reset_password.txt', user=user),
               html_body=flask.render_template('email/reset_password.html', user=user)

But the problem is, we can't identify if the person that is requesting is actually the user, we need to use a token. A token is a little encoded string, that will only belong to the user.<Br>
<a href="https://pyjwt.readthedocs.io/en/latest/">`pyJWT`</a> is a library that provide some functions to deal with tokens, with this library, we can encode some data with a key.
> JWT stands for <a href="https://en.wikipedia.org/wiki/JSON_Web_Token">Json Web Token</a>
    
The key we will use to encode the payload will be our secret key.<br>
We need the `jwt.encode` function to create an encoded token.<br>
Let's implement a function that create a token in our user model:<br>
<br>
<small>In User model:</small>

In [ ]:
import time
import jwt
#class User(...):
    #...
    def get_reset_password_token(self, expires_in=600):
        timeout = time.time() + expires_in
        payload = {
            'reset_password': self.id,
            'exp': timeout
        }
        
        # Get the secret key from config
        secret_key = app.config['SECRET_KEY']
        
        # Create the token
        token = jwt.encode(payload, secret_key, algorithm="HS256")
        
        # Turn it to string
        s_token = token.decode('utf-8')
        
        return s_token


We can now update our `send_password_reset_email` function

In [ ]:
def send_password_reset_email(user):
    token = user.get_reset_password_token()
    send_email('Password reset',
               sender="mymail@gmail.com",
               recipients=[user.email],
               text_body=flask.render_template('email/reset_password.txt', user=user, token=token),
               html_body=flask.render_template('email/reset_password.html', user=user, token=token))

This token is supposed to allow us to identify the user that its's linked to.<Br>
We also need to add a function to verify this token.<br>
<br>
<small>In the User model:</small>


In [ ]:
import time
import jwt

#class User(...):
    #...
    def get_reset_password_token(self, expires_in=600):
        timeout = time.time() + expires_in
        payload = {
            'reset_password': self.id,
            'exp': timeout
        }
        
        # Get the secret key from config
        secret_key = app.config['SECRET_KEY']
        
        # Create the token
        token = jwt.encode(payload)
        
        # Turn it to string
        s_token = token.decode('utf-8')
        
        return s_token

    def verify_reset_password_token(self, token):
        try:
            id = jwt.decode(token, app.config['SECRET_KEY'],
                            algorithms=['HS256'])['reset_password']
        except:
            return
        return User.query.get(id)

### Now, let's create the templates for the function

Let's create the two templates for text and html body
<br>
<small>reset_password.txt:</small>

And the html template
<br>
<small>reset_password.html:</small>

### Step 2: let's create a form to request the mail

This page requires a form, so we need a form, a template and a route:<br>
<br>
<small>Form:</small>

In [ ]:
class ResetPasswordRequestForm(FlaskForm):
    email = StringField('Email', validators=[DataRequired(), Email()])
    submit = SubmitField('Request Password Reset')

<small>Template:</small>

<small>Route:</small>

In [ ]:
from app import forms, mailing

@app.route('/reset_password_request', methods=['GET', 'POST'])
def reset_password_request():
    
    if current_user.is_authenticated:
        return redirect(url_for('index'))
    
    form = forms.ResetPasswordRequestForm()
    
    if form.validate_on_submit():
        user = User.query.filter_by(email=form.email.data).first()
        
        if user:
            mailing.send_password_reset_email(user)
            
        flash('An email has been sent')  
        return redirect(url_for('login'))
    return render_template('reset_password_request.html', title='Reset Password', form=form)

### Step 3: When the user has clicked on the mail, he needs to get to a Reset password form

Form:

In [ ]:
class ResetPasswordForm(FlaskForm):
    password = wtforms.PasswordField('Password', validators=[DataRequired()])
    confirm  = wtforms.PasswordField('Confirm Password', validators=[validators.DataRequired(), validators.EqualTo('password')])
    submit = wtforms.SubmitField('Request Password Reset')

Template:

# That's it !

---

> You can send your mail as a thread if it slows down your application